# Implied Volatility

Implied volatility is a key concept in options trading and is a measure of the market's expectations of the future volatility of the underlying asset's price. It is derived from the market price of an option and reflects the market's consensus about the future price movements of the underlying asset.

## Basic Concept of Volatility

- **Historical Volatility:** This measures the actual past volatility **of the asset**, calculated based on historical price data.
- **Implied Volatility (IV):** This represents the market's forecast of a stock's future volatility. Unlike historical volatility, **implied volatility is forward-looking** and is derived from the prices **of options on the asset**.

## Why Implied Volatility is Important

- **Market Sentiment:** IV reflects the market's consensus about the future volatility of the asset. High IV indicates that the market expects significant price movements, while low IV suggests relatively stable prices.
- **Pricing and Strategy:** Traders use IV to gauge the attractiveness of option prices and to develop trading strategies. For instance, if a trader believes the market's implied volatility is too high or too low relative to their own forecast, they may execute trades to exploit this discrepancy.
- **Risk Management:** Understanding IV helps traders manage the risk associated with options positions and to make informed decisions about entering or exiting trades.

## Finding Implied Volatility

Implied volatility is not directly observable. Instead, it must be inferred from the market price of an option using models like Black-Scholes. This process involves the following steps:

1. **Market Price of the Option:** Obtain the current market price of the option.
2. **Inputs for Black-Scholes Model:** Gather the necessary inputs (current stock price, strike price, time to expiration, risk-free rate).
3. **Iterative Process:** Use numerical methods (like the Newton-Raphson method) to iteratively solve the Black-Scholes model for $ \sigma $ (implied volatility) that makes the theoretical price equal to the market price ($10).

# Newton-Raphson Method

Calculating implied volatility can be done using several methods, but the most commonly used method is the Newton-Raphson due to its quadratic convergence properties, which makes it faster than many other root-finding algorithms. However, it requires the function to be differentiable and a good initial guess to ensure convergence.


## Steps of the Newton-Raphson Method

1. **Initial Guess:** Choose an initial guess $ x_0 $.
2. **Compute the Function Value and Derivative:** Evaluate $ f(x_n) $ and $ f'(x_n) $.
3. **Update the Approximation:** Compute the next approximation using $ x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)} $.
4. **Check for Convergence:** If $ |x_{n+1} - x_n| $ is less than a predefined tolerance, stop the iteration. Otherwise, set $ x_n = x_{n+1} $ and repeat from step 2.

## Advantages

- **Fast Convergence:** The method converges quickly when close to the root, especially if the function behaves well (smooth and continuous).
- **Simple to Implement:** The algorithm is straightforward to code and requires only basic calculus (evaluation of the function and its derivative).

## Disadvantages

- **Initial Guess Dependency:** A poor initial guess can lead to slow convergence or divergence.
- **Derivative Requirement:** The method requires the computation of the derivative of the function, which may not always be easy or possible.
- **Not Globally Convergent:** The method is not guaranteed to converge for all functions or all initial guesses.

# Calculating Implied Volatility using Newton-Raphson Method

## The Black-Scholes Formula

The Black-Scholes formula for a call option is:
<br>
<br>
$$ C = S_0 N(d_1) - K e^{-rT} N(d_2)$$
<br>
The Black-Scholes formula for a put option is:
<br>
<br>
$$ P = K e^{-rT} N(-d_2) - S_0 N(-d_1)$$
<br>
where:
<br>
$$ d_1 = \frac{\ln(S_0 / K) + (r + \sigma^2 / 2)T}{\sigma \sqrt{T}}$$
<br>
$$ d_2 = d_1 - \sigma \sqrt{T}$$
<br>
where:
- $ C $ = Call option price
- $ P $ = Put option price
- $ S_0 $ = Current price of the underlying asset
- $ K $ = Strike price
- $ T $ = Time to expiration
- $ r $ = Risk-free interest rate
- $ \sigma $ = Volatility (the parameter we need to find)
- $ N(d) $ = Cumulative distribution function of the standard normal distribution

## 2. Formulate the Problem

Given the market price of the option $ C_{\text{market}} $ or $ P_{\text{market}} $, we need to find $ \sigma $ such that the Black-Scholes price $ C(\sigma) $ or $ P(\sigma) $ matches the market price.
<br>
<br>
For call options:

$$ f(\sigma) = C(\sigma) - C_{\text{market}}$$

For put options:

$$ f(\sigma) = P(\sigma) - P_{\text{market}}$$
<br>
Our goal is to find the root of $ f(\sigma) = 0 $.

## 3. Newton-Raphson Iterative Formula

The Newton-Raphson method updates the guess for $ \sigma $ using:

$$ \sigma_{n+1} = \sigma_n - \frac{f(\sigma_n)}{f'(\sigma_n)}$$

## 4. Derivative of the Black-Scholes Formula

To apply the Newton-Raphson method, we need the derivative $ f'(\sigma) $, which is given by the Vega ($ \nu $) of the option:

$$ \nu = S_0 \sqrt{T} N'(d_1)$$
<br>
where $ N'(d_1) $ is the probability density function of the standard normal distribution evaluated at $ d_1 $.

## 5. Iterative Algorithm

1. **Initial Guess:** Start with an initial guess $ \sigma_0 $. A common choice is the historical volatility of the underlying asset.
2. **Iterate:**
   - Compute $ C(\sigma_n) $ or $ P(\sigma_n) $ using the Black-Scholes formula.
   - Compute $ \nu(\sigma_n) $.
   - Update $ \sigma_{n+1} = \sigma_n - \frac{C(\sigma_n) - C_{\text{market}}}{\nu(\sigma_n)} $ or $ \sigma_{n+1} = \sigma_n - \frac{P(\sigma_n) - P_{\text{market}}}{\nu(\sigma_n)} $.
   - Repeat until $ |C(\sigma_{n+1}) - C_{\text{market}}| $ or $ |P(\sigma_{n+1}) - P_{\text{market}}| $ is within a predefined tolerance level.

## 6. Implementation

In [ ]:
import numpy as np
from scipy.stats import norm

In [ ]:
def calc_d1(S, K, T, r, sigma):
    return (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))

def calc_d2(d1, sigma, T):
    return d1 - sigma * np.sqrt(T)

def black_scholes(S, K, T, r, sigma, opt_type):
    assert opt_type in ["c", "p"], "Invalid option type. Choose 'c' for 'call' or 'p' for 'put'."

    d1 = calc_d1(S, K, T, r, sigma)
    d2 = calc_d2(d1, sigma, T)

    # Calculate the price of the option
    if opt_type == "c":
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

    return price

def calc_vega(S, K, T, r, sigma):
    d1 = calc_d1(S, K, T, r, sigma)
    vega = S * np.sqrt(T) * norm.pdf(d1, 0, 1)

    return vega

def implied_volatility(market_price, S, K, T, r, sigma_init, opt_type, tol=1e-5, max_iterations=100):
    sigma = sigma_init
    for i in range(max_iterations):
        bs_price = black_scholes(S, K, T, r, sigma, opt_type)
        v = calc_vega(S, K, T, r, sigma)
        sigma_new = sigma - (bs_price - market_price) / v
        if abs(sigma_new - sigma) < tol:
            return sigma_new
        sigma = sigma_new
    return sigma

In [ ]:
# Parameters
S = 30
K = 28
T = 0.5
r = 0.025
C_market = 5
sigma_init = 0.2
opt_type = "c"

In [ ]:
# Calculate implied volatility
iv = implied_volatility(C_market, S, K, T, r, sigma_init, opt_type)
print(f"Implied Volatility: {iv*100:.2f}")

Implied Volatility: 45.83
